<a href="https://cognitiveclass.ai/"><img src = "https://compete.cognitiveclass.ai/static/media/cognitive-class-logo.b08236c1.png" width = 384></a>

# Segmenting and Clustering Boroughs in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

1. <a href="#item1">Introduction</a>

2. <a href="#item2">Load and Explore Dataset</a>

3. <a href="#item3">Explore Boroughs in Toronto</a>

4. <a href="#item4">Analyze Each Borough</a>

5. <a href="#item5">Cluster Boroughs</a>

6. <a href="#item6">Examine Clusters</a>

</font>
</div>

## 1. Introduction

For this assignment, Foursquare API will be used to explore boroughs in Toronto. The *API* to get the most common venue categories in each borough is going to be used for exploring, and then use this feature to group the boroughs into clusters. K-means clustering algorithm will be applied to complete the clusterization. The **Folium** library is used for borough visualization the city of Toronto and their emerging clusters.

Before loading the data and exploring it, download and import all the dependencies that are needed for this assignment.

In [1]:
import numpy as np
import pandas as pd
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

!pip install folium

print("All dependencies successfully installed!")

All dependencies successfully installed!


## 2. Load and Explore Dataset

Toronto has a total of **11** boroughs and **103** neighborhoods. In order to segment the boroughs and explore them, a dataset with all the boroughs and the neighborhoods that exist in each borough, as well as the the latitude and longitude coordinates of each borough, is loaded.

This dataset has been built from the previous assignments, so all that needs to be done, is to load *CSV* into a Pandas dataframe.

#### Load and explore the data

In [2]:
df = pd.read_csv("toronto_neighborhood_coordinates.csv", index_col = 0)

Let's take a quick look at the data.

In [3]:
df.iloc[0]

PostalCode                 M1B
Borough            Scarborough
Neighborhood    Rouge, Malvern
Latitude               43.7976
Longitude              -79.227
Name: 0, dtype: object

In [4]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.7976,-79.2270
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7882,-79.1911
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7385,-79.2021
3,M1G,Scarborough,Woburn,43.7563,-79.2224
4,M1H,Scarborough,Cedarbrae,43.7563,-79.2417


Make sure that the dataset has all **11** boroughs and **103** neighborhoods.

In [5]:
print("The dataframe has {} boroughs and {} neighborhoods.".format(len(df["Borough"].unique()), df.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


#### Create a map of Toronto with boroughs superimposed on top

In [6]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location = [43.7241346, -79.4174509], zoom_start = 10)

# Add markers to map
for lat, long, borough, neighborhood in zip(df["Latitude"], df["Longitude"], df["Borough"], df["Neighborhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long],
                        radius = 5.0,
                        weight = 1.5,
                        popup = label,
                        color = "#FF9900",
                        fill = True,
                        fill_color = "#FFFF00",
                        fill_opacity = 0.5,
                        parse_html = False).add_to(map_toronto)
map_toronto

Simplify the above map and segment and cluster only the boroughs in Toronto area. Slice the original dataframe and create a new dataframe of the Toronto area data.

In [7]:
toronto_df = df[df["Borough"].str.match(".*Toronto")].reset_index(drop = True)
toronto_df.head(16)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6675,-79.2960
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6668,-79.3501
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6620,-79.3281
3,M4M,East Toronto,Studio District,43.6505,-79.3369
4,M4N,Central Toronto,Lawrence Park,43.7168,-79.3998
5,M4P,Central Toronto,Davisville North,43.7066,-79.3980
6,M4R,Central Toronto,North Toronto West,43.7066,-79.3996
7,M4S,Central Toronto,Davisville,43.6964,-79.3953
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6825,-79.3897
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6778,-79.3992


View the unique boroughs that were retrieved for Toronto area.

In [8]:
for idx, borough in enumerate(toronto_df["Borough"].unique()):
    print(str(idx + 1) + ". " + borough)

1. East Toronto
2. Central Toronto
3. Downtown Toronto
4. West Toronto


As it was done with all of city of Toronto, visualize only the boroughs from Toronto main area.

In [9]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location = [43.7241346, -79.4174509], zoom_start = 11)

# Add markers to map
for lat, long, neighborhood in zip(toronto_df["Latitude"], toronto_df["Longitude"], toronto_df["Neighborhood"]):
    label = "{}".format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long],
                        radius = 7.5,
                        weight = 1.5,
                        popup = label,
                        color = "#2200DD",
                        fill = True,
                        fill_color = "#2222FF",
                        fill_opacity = 0.5,
                        parse_html = False).add_to(map_toronto)
map_toronto

#### <font color = "#FF3322">Important!</font>

Even though the above map shows only the boroughs for the main area of Toronto, for further analysis, all the boroughs will be used, because the main area of Toronto has only **four** boroughs, which is not enough for an accurate clusterization.

## 3. Explore Boroughs in Toronto

#### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = "********"
CLIENT_SECRET = "********"
VERSION = "********"

print("Your credentails:")
print("Client ID:", CLIENT_ID)
print("Client Secret:", CLIENT_SECRET)

Your credentails:
Client ID: ********
Client Secret: ********


#### Create a function to retrieve the venues for all the boroughs in Toronto area

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500, limit = 100):
    venues_list = []

    for name, lat, long in zip(names, latitudes, longitudes):

        # Create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            long,
            radius,
            limit)

        # Make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]

        # Return only relevant information for each nearby venue
        venues_list.append([(name,
                             lat,
                             long,
                             v["venue"]["name"],
                             v["venue"]["location"]["lat"],
                             v["venue"]["location"]["lng"],
                             v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Borough",
                             "Borough Latitude",
                             "Borough Longitude",
                             "Venue",
                             "Venue Latitude",
                             "Venue Longitude",
                             "Venue Category"]
    return(nearby_venues)

#### Test the function with a random borough from our dataset

Get a random sample from the data set and get the nearby venues for that borough

In [12]:
# Call the `sample` method for extracting a random row from the dataframe
sample = df.sample().iloc[0]

# View the extracted
sample

PostalCode                              M4T
Borough                     Central Toronto
Neighborhood    Moore Park, Summerhill East
Latitude                            43.6825
Longitude                          -79.3897
Name: 48, dtype: object

In [13]:
# Retrieve nearby venues for the above sample
sample_venues = getNearbyVenues(names = [sample["Borough"]],
                                latitudes = [sample["Latitude"]],
                                longitudes = [sample["Longitude"]])
sample_venues.head(16)

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Toronto,43.6825,-79.3897,LCBO,43.681497,-79.391261,Liquor Store
1,Central Toronto,43.6825,-79.3897,Boxcar Social,43.681750,-79.391447,Coffee Shop
2,Central Toronto,43.6825,-79.3897,Waxon Waxbar,43.682824,-79.391971,Spa
3,Central Toronto,43.6825,-79.3897,Manipedi Spa,43.682626,-79.391879,Spa
4,Central Toronto,43.6825,-79.3897,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
5,Central Toronto,43.6825,-79.3897,kibo sushi house,43.682727,-79.392270,Sushi Restaurant
6,Central Toronto,43.6825,-79.3897,nutbar,43.682811,-79.391945,Vegetarian / Vegan Restaurant
7,Central Toronto,43.6825,-79.3897,The Rosedale Diner,43.680448,-79.390816,Wine Bar
8,Central Toronto,43.6825,-79.3897,Terroni,43.679870,-79.390525,Italian Restaurant
9,Central Toronto,43.6825,-79.3897,Greenhouse Juice Co,43.679101,-79.390686,Juice Bar


#### Run the above function on each borough and create a new dataframe called with the retrieved venues

In [14]:
toronto_venues = getNearbyVenues(names = df["Borough"],
                                 latitudes = df["Latitude"],
                                 longitudes = df["Longitude"])

#### Check the preview and size of the resulting dataframe

In [15]:
print("Size:", toronto_venues.shape)

toronto_venues.head()

Size: (2811, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.7976,-79.227,TD Canada Trust,43.796017,-79.226101,Bank
1,Scarborough,43.7976,-79.227,Barrio Fiesta,43.795757,-79.226644,Restaurant
2,Scarborough,43.7976,-79.227,Pizza Pizza,43.795986,-79.226569,Pizza Place
3,Scarborough,43.7976,-79.227,Bun King Bakery,43.795995,-79.226498,Bakery
4,Scarborough,43.7976,-79.227,Burrows Hall Park,43.793712,-79.226704,Baseball Field


View how many venues were returned for each borough

In [16]:
toronto_venues.groupby("Borough").count()[["Venue"]]

,Venue
Borough,
Central Toronto,438
Downtown Toronto,1285
East Toronto,135
East York,60
Etobicoke,150
Mississauga,11
North York,233
Queen's Park,37
Scarborough,112


#### Find out how many unique categories can be curated from all the returned venues

In [17]:
print("There are {} uniques categories.".format(len(toronto_venues["Venue Category"].unique())))

There are 271 uniques categories.


## 4. Analyze Each Borough

In [18]:
# One-hot encoding
toronto_one_hot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix = "", prefix_sep = "")

# Add borough column back to dataframe
toronto_one_hot["Borough"] = toronto_venues["Borough"] 

# Move borough column to the first column
fixed_columns = [toronto_one_hot.columns[-1]] + list(toronto_one_hot.columns[:-1])
toronto_one_hot = toronto_one_hot[fixed_columns]

toronto_one_hot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Examine the new dataframe size.

In [19]:
toronto_one_hot.shape

(2811, 272)

#### Group rows by borough by taking the mean of the frequency of occurrence for each category

In [20]:
toronto_grouped = toronto_one_hot.groupby("Borough").mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.002283,0.000000,0.009132,0.000000,0.002283,0.002283,0.004566,0.009132,0.002283,0.004566,0.000000,0.002283,0.009132,0.009132,0.

#### Confirm the new size

In [21]:
toronto_grouped.shape

(11, 272)

#### Print each borough along with the top **five** most common venues

In [22]:
top_venues_count = 5

for borough in toronto_grouped["Borough"]:
    print("----" + borough + "----")

    row = toronto_grouped[toronto_grouped["Borough"] == borough].T.reset_index()
    row.columns = ["Venue", "Frequency"]
    row = row.iloc[1:]
    row["Frequency"] = row["Frequency"].astype(float)
    row = row.round({"Frequency": 2})

    print(row.sort_values("Frequency", ascending = False).reset_index(drop = True).head(top_venues_count))
    print("\n")

----Central Toronto----
                Venue  Frequency
0         Coffee Shop       0.08
1  Italian Restaurant       0.05
2                Café       0.03
3                 Pub       0.03
4    Sushi Restaurant       0.03


----Downtown Toronto----
              Venue  Frequency
0       Coffee Shop       0.09
1              Café       0.05
2        Restaurant       0.04
3             Hotel       0.03
4  Sushi Restaurant       0.02


----East Toronto----
                   Venue  Frequency
0            Coffee Shop       0.06
1   Fast Food Restaurant       0.04
2                    Bar       0.04
3                 Bakery       0.04
4  Vietnamese Restaurant       0.04


----East York----
            Venue  Frequency
0     Coffee Shop       0.07
1            Park       0.05
2   Grocery Store       0.05
3            Café       0.05
4  Sandwich Place       0.05


----Etobicoke----
                 Venue  Frequency
0          Coffee Shop       0.09
1                 Park       0.05
2  Sportin

#### Put the results into a *Pandas* dataframe

Define a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now create a new dataframe and display the top **ten** venues for each borough.

In [24]:
top_venues_count = 10

indicators = ["st", "nd", "rd"]

# Create columns according to number of top venues
columns = ["Borough"]

for idx in np.arange(top_venues_count):
    try:
        columns.append("{}{} Most Common Venue".format(idx + 1, indicators[idx]))
    except:
        columns.append("{}th Most Common Venue".format(idx + 1))

# Create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns = columns)
boroughs_venues_sorted["Borough"] = toronto_grouped["Borough"]

for idx in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[idx, 1:] = return_most_common_venues(toronto_grouped.iloc[idx, :], top_venues_count)

boroughs_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bar,Bakery,American Restaurant,Japanese Restaurant,Gastropub
2,East Toronto,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
3,East York,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
4,Etobicoke,Coffee Shop,Park,Restaurant,Grocery Store,Sporting Goods Shop,Bakery,Pharmacy,Bank,Department Store,Sandwich Place
5,Mississauga,Hotel,Coffee Shop,Middle Eastern Restaurant,Burrito Place,American Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Sandwich Place,Fried Chicken Joint,Filipino Restaurant
6,North York,Coffee Shop,Bank,Pharmacy,Vietnamese Restaurant,Pizza Place,Park,Café,Grocery Store,Italian Restaurant,Spa
7,Queen's Park,Coffee Shop,Gym,Italian Restaurant,Nightclub,Japanese Restaurant,Diner,College Theater,Sushi Restaurant,Liquor Store,Dance Studio
8,Scarborough,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
9,West Toronto,Bakery,Italian Restaurant,Restaurant,Café,Coffee Shop,Bar,Park,Gym / Fitness Center,Pizza Place,Yoga Studio


## 5. Cluster Boroughs

Run <i>K</i>-means to cluster the boroughs into <b>five</b> clusters.

In [25]:
# Set number of clusters
k_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop("Borough", 1)

# Run K-means clustering
kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 3, 0, 1, 0, 4, 0, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top **ten** venues for each neighborhood.

In [26]:
# Add clustering labels
boroughs_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

toronto_merged = df

# Merge Toronto grouped data with toronto data to add latitude and longitude for each borough
toronto_merged = toronto_merged.join(boroughs_venues_sorted.set_index("Borough"), on = "Borough")

toronto_merged.sample(8)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,M4J,East York,East Toronto,43.6713,-79.3412,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
61,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6492,-79.3823,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bar,Bakery,American Restaurant,Japanese Restaurant,Gastropub
81,M6N,York,"The Junction North, Runnymede",43.6680,-79.4515,2,Pizza Place,Bakery,Café,Coffee Shop,Latin American Restaurant,Italian Restaurant,Sandwich Place,Grocery Store,Food & Drink Shop,Thai Restaurant
43,M4M,East Toronto,Studio District,43.6505,-79.3369,2,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
73,M6C,York,Humewood-Cedarvale,43.6830,-79.4184,2,Pizza Place,Bakery,Café,Coffee Shop,Latin American Restaurant,Italian Restaurant,Sandwich Place,Grocery Store,Food & Drink Shop,Thai Restaurant
70,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.6492,-79.3823,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Bar,Bakery,American Restaurant,Japanese Restaurant,Gastropub
20,M2L,North York,"Silver Hills, York Mills",43.7352,-79.3818,0,Coffee Shop,Bank,Pharmacy,Vietnamese Restaurant,Pizza Place,Park,Café,Grocery Store,Italian Restaurant,Spa
93,M9A,Etobicoke,Islington Avenue,43.6434,-79.5297,0,Coffee Shop,Park,Restaurant,Grocery Store,Sporting Goods Shop,Bakery,Pharmacy,Bank,Department Store,Sandwich Place


Visualize the resulting clusters.

In [27]:
# Create map
map_clusters = folium.Map(location = [43.7241346, -79.4174509], zoom_start = 10)

# Set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i * x) ** 2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(toronto_merged["Latitude"], toronto_merged["Longitude"], toronto_merged["Borough"], toronto_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + ", cluster " + str(cluster), parse_html = True)

    folium.CircleMarker([lat, long],
                        radius = 5.0,
                        weight = 1.5,
                        popup = label,
                        color = rainbow[cluster - 1],
                        fill = True,
                        fill_color = rainbow[cluster - 1],
                        fill_opacity = 0.5).add_to(map_clusters)
map_clusters

## 6. Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, assign a name to each cluster.

#### Cluster 0

In [28]:
toronto_merged.loc[toronto_merged["Cluster Labels"] == 0,
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
1,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
2,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
3,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
4,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
5,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
6,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
7,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
8,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery
9,Scarborough,0,Park,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Pharmacy,Restaurant,Bank,Bakery


#### Cluster 1

In [29]:
toronto_merged.loc[toronto_merged["Cluster Labels"] == 1,
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Mississauga,1,Hotel,Coffee Shop,Middle Eastern Restaurant,Burrito Place,American Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Sandwich Place,Fried Chicken Joint,Filipino Restaurant


#### Cluster 2

In [30]:
toronto_merged.loc[toronto_merged["Cluster Labels"] == 2,
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
41,East Toronto,2,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
42,East Toronto,2,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
43,East Toronto,2,Coffee Shop,Bakery,Vietnamese Restaurant,Fast Food Restaurant,Bar,Pizza Place,Chinese Restaurant,Breakfast Spot,Grocery Store,Restaurant
44,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
45,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
46,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
47,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
48,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant
49,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pub,Grocery Store,Restaurant,French Restaurant,Thai Restaurant,Japanese Restaurant


#### Cluster 3

In [31]:
toronto_merged.loc[toronto_merged["Cluster Labels"] == 3,
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
36,East York,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
38,East York,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
39,East York,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant
40,East York,3,Coffee Shop,Grocery Store,Café,Park,Sandwich Place,Trail,Burger Joint,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant


#### Cluster 4

In [32]:
toronto_merged.loc[toronto_merged["Cluster Labels"] == 4,
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Queen's Park,4,Coffee Shop,Gym,Italian Restaurant,Nightclub,Japanese Restaurant,Diner,College Theater,Sushi Restaurant,Liquor Store,Dance Studio


It is noticeable in the above analysis that every cluster is rich in **Coffee Shop** venues, as it is expected, since it is one of the most common venues in large cities.

A name will be assigned to each cluster based on the first **three** most common venues:
1. Cluster 0: **The leisure area**, since it is rich in parks.
2. Cluster 1: **The hotel area**, since it is rich hotels.
3. Cluster 2: **The dinning area**, since it is rich various restaurants with different cuisines.
4. Cluster 3: **The grocery shopping area**, since it is rich grocery stores.
5. Cluster 4: **The gym area**, since it is rich gym.

Besides the above naming, every cluster is quite in rich in **Coffee Shops**.